In [1]:
import torch
from BPE_Encoder import  token2id, detokenise,encode_prompt
from Transformer import MLLM

In [2]:
best_parameters = {'lr': 0.0008846917512346465, 'weight_decay': 0.00029475907910799643, 'dropout_percentage': 0.0010306170289606452, 'heads': 4, 'num_layers': 6, 'dim': 64, 
 'label_smoothing': 0.019019051540349757, 'pct_start': 0.35980068988151803, 'activation': 'gelu', 'ffn_internal': 4}

In [ ]:
model = MLLM.load_from_checkpoint(
        "E:/Projects/Seq2Seq/lightning_logs/version_0/checkpoints/best.ckpt",
        vocab=len(token2id),
        dim=best_parameters["dim"],
        pad_idx=token2id["<pad>"],
        max_pos=512,
        QKV_dim=best_parameters["dim"] // best_parameters["heads"],
        heads=best_parameters["heads"],
        num_layers=best_parameters["num_layers"],
        dropout_percentage=best_parameters["dropout_percentage"],
        learning_rate=best_parameters["lr"],
        wd=best_parameters["weight_decay"],
        ls=best_parameters["label_smoothing"],pct_start = best_parameters['pct_start'], act=best_parameters['activation'],
        ffn_internal = best_parameters['ffn_internal'])   
model.eval().to('cuda')                 


In [9]:
def generate(prompt: str,
             max_new_tokens: int = 500,
             temperature: float = 0.7,
             top_k: int | None = 20) -> str:

    input_ids = encode_prompt(prompt).unsqueeze(0).to('cuda')
    generated = input_ids

    with torch.no_grad():
        for _ in range(max_new_tokens):
            if generated.size(1) >= 1024:          
                generated = generated[:, -1023:]   
            logits = model(generated)[:, -1, :] / temperature
            if top_k is not None:
                values, _ = torch.topk(logits, top_k)
                logits = torch.where(
                    logits < values[:, [-1]],
                    torch.full_like(logits, -1e10),
                    logits,
                )
            probs   = torch.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            if next_id.item() == token2id["<eos>"]:
                break
            generated = torch.cat([generated, next_id], dim=1)
    return detokenise(generated[0].tolist())


# example usage
prompt = "logistic regression is best for"
print(generate(prompt, temperature=0.7, top_k=20))


logistic regression is best for the regression and the lasso and regression


In [1]:
vocab = []
with open("E:/Projects/Seq2Seq/data/corpus/Vocab.txt", "r", encoding="utf-8") as f:
    for tok in f:
        vocab.append(tok.strip())

In [2]:
len(vocab)

20030